In [1]:
import os
from  elasticsearch import Elasticsearch
import glob
import pandas as pd
from py_linq import Enumerable
import json
import re




In [4]:
path= r"C:/Users/nerdi/python/dict/"

In [5]:
es = Elasticsearch()

In [ ]:
index="ipa_dict"

In [ ]:
allstuff= []

In [ ]:
mapping={
    "mappings":{
        "properties":{
            "phones":{
                "type": "text"
            },
            "word":{
                "type":"keyword"
            }
        }
    }
}

In [ ]:
#run only once
es.indices.create(index=index ,ignore= 400)

In [ ]:
files =os.listdir(path)

In [ ]:
for f in files:
    dataTemp = pd.read_csv(path+'\\'+f, sep="\t", header= None)
    dataTemp['locale'] = f[:-4]
    #pd.concat(allstuff, ignore_index=True)
    allstuff.append(dataTemp)
    

In [ ]:
allstuff = pd.concat(allstuff, ignore_index=True)   

In [ ]:
## Elastic Query building 
def search_in_dict(word):
    qs= {
          "query": {
            "bool": {
              "should": [
                {
                  "query_string": {
                    "query": word
                  }
                }
              ]
            }
          }
        }
    resp  = es.search(index=index, 
                  body=qs
                 )
    if resp:
        #print(resp["hits"]["hits"])
        return resp["hits"]["hits"]
    else:
        print("nothing found")
        return False

In [ ]:
def insert_new(nWord, nLocale, nPhones):
    print("inserting")
    doc = {
    'word': nWord,
    'phones':[nPhones],
    'locales':[
        { nLocale: nPhones}    
    ]}
    print(doc)
    res = es.index(index=index, body=doc)
    print("inserted",res)
    


In [ ]:
def get_next_key(doc, key):
    new_key = key
    next_var = 1
    exists = False
    for x in doc["locales"]:
        if key in x.keys():
            exists = True
    if not exists:
        return key
    try:
        current_var = int(re.search(r'\d+', key).group())
        #print(current_var)
        next_var = current_var + 1
        new_key = key.removesuffix(str(current_var)) + str(next_var)
    except:
        new_key = key + str(next_var)

    for x in doc["phones"]:
        if key in x.keys():
            new_key = get_next_key(doc, new_key)
    return new_key


In [ ]:
def update_existing(_currentModel, _locale, _phones):
    print("updating")
    doc = _currentModel["_source"]
    print(doc)
    for x in doc["phones"]:
        if x[0] ==phones:
            return
        print ("x ", x[0])
        for y in x.keys():
            print("y", y)
            if y == _phones:
                return                
  
    _locale = get_next_key(doc, _locale)
    doc["phones"].append(_phones)   
    doc["locales"].append({_locale:_phones}) 
    #print(doc)    
    res = es.index(index=index, id=_currentModel["_id"], body=doc)

In [ ]:
#for every word in df
    # set word to lower
def insert_or_update(w):
    try:
        nWord = w[0].lower()
        nPhones = w[1]
        nPhones = nPhones.removesuffix('/')
        nPhones = nPhones.removeprefix('/')
        nLocale = w["locale"]
        
        hits = search_in_dict(nWord)
        if hits:
            for x in hits:
                if x["_source"]["word"] == nWord:
                    update_existing(x, nLocale, nPhones)
                    # update not insert
                    found = True
                    break
        else:
            insert_new(nWord, nLocale, nPhones)
    except:
        print("error",w)


In [ ]:
allstuff

In [ ]:
allstuff.apply(lambda x: insert_or_update(x), axis=1)

In [ ]:
allstuff[1:2]

In [ ]:
single=allstuff[1:2]

In [ ]:
single[1]

In [ ]:
single[0].lower()


In [ ]:
insert_or_update(single)

In [ ]:
type(single)

In [ ]:
asd = str(single[1])

In [ ]:
asd

In [ ]:
asd.lower()

In [ ]:
asd= asd.removesuffix("/")

In [ ]:
asd= asd.removeprefix("/")

In [ ]:
insert_or_update(single)

In [ ]:
single[1].removesuffix('/')

In [ ]:
allstuff[0][1:2][1].removesuffix('/')